In [3]:
import numpy as np
import pandas as pd

#РАСПРЕД ПО СПИСКУ МАГОВ
#список складов для распреда
all_llist = [4836,4840,4841,4834,4835,4827,4826,4817,4824,4805,4822]
#all_llist.reverse()

#prih1 = pd.read_csv(r'C:\Users\Dotsenko.Semen\Documents\ItemMoves.csv', delimiter=';', encoding='cp1251',usecols=list(range(0, 48)))
#del prih1['№ Поставщика']
#ИЛИ
prih1 = pd.read_excel(r'C:\рушники\123.xlsx')

prih1 = prih1.sort_values(by='Товар')

# словарь тов - количество и список товов
tov_list = prih1.Товар.unique().tolist()
tov_dict = prih1.Товар.value_counts()[prih1.Товар.unique()].to_dict()

# остатки
stock = pd.read_excel(r'C:\Остатки\обручи.xlsx', usecols=['КодСклада','ТоварНо'])
#stock = pd.read_excel(r'C:\Остатки\остатки.xlsx', usecols=['КодСклада','ТоварНо'])
stock = stock[stock['КодСклада'].isin(all_llist) & stock['ТоварНо'].isin(tov_list)]

# таблица остатки товов по складам
pivot = pd.pivot_table(stock,index='КодСклада', columns='ТоварНо', aggfunc=len, fill_value=0).reset_index()
pivot = pd.DataFrame({'КодСклада':all_llist}).merge(pivot,how='left')

# добавим в таблицу колонки с нулями по товам, которых не хватает
add_cols =  [x for x in tov_list if x not in pivot.columns.to_list()]
add_zeroes = [list(np.zeros(len(all_llist),dtype='int'))]*len(add_cols)

pivot = pivot.reindex(columns=pivot.columns.tolist() + add_cols)

pivot.fillna(0,inplace=True)
pivot = pivot.astype('int')
pivot = pivot[['КодСклада'] + tov_list]

# Если количество по тову в прихе больше чем в списке то к списку добавить нули, если нет то обрезать
sklad_list = [
    pivot[pivot[x]==0]['КодСклада'].tolist() + 
    list(np.zeros((tov_dict[x] - len(pivot[pivot[x]==0]['КодСклада'].tolist())),dtype='int')) 
    if 
    tov_dict[x] > len(pivot[pivot[x]==0]['КодСклада'].tolist()) 
    else pivot[pivot[x]==0]['КодСклада'].tolist()[:tov_dict[x]] 
    for x in tov_list
]

sklad_list = [item for sublist in sklad_list for item in sublist]

prih1['Период реализации'] = sklad_list

prih1 = prih1[prih1['Период реализации'] > 0]

# Сохраняем в ексель на лист "движение товара"

import random
name = 'C:/рушники/' + str(random.randrange(5000,10000)) + '.xlsx'
prih1.to_excel(name, sheet_name='Движение товара', index=False)

In [4]:
prih1[['Период реализации','Склад']].groupby(by='Период реализации').count().sort_values(by='Склад',ascending=False)

,Склад
Период реализации,
4836,17
